# Module 4: Visual Lane Servoing and Lane Localization

Now that we have our basic motion and camera imaging that we can get from our robot, we want to combine the two where it can travel down a road while staying within the lanes.

This Module should follow Module 5: Road Sign Detection

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time

robot = Robot()
camera = Camera.instance()

In [ ]:
def detect_lane_markings(image: np.ndarray):
    """
    Args:
        image: An image from the robot's camera in the BGR color space (numpy.ndarray)
    Return:
        left_masked_img:   Masked image for the dashed-yellow line (numpy.ndarray)
        right_masked_img:  Masked image for the solid-white line (numpy.ndarray)
    """
    h, w, _ = image.shape
    # print(image.shape)

    imgbgr = image

    # Convert the image to HSV for any color-based filtering
    imghsv = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2HSV)

    # Most of our operations will be performed on the grayscale version
    imggray = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2GRAY)

    horizon_X = 130
    mask_ground = np.zeros((h, w), dtype=np.uint8)
    mask_ground[int(h - horizon_X) :, :] = 1
    # print(h," " ,w," " ,int(h - horizon_X[0]))

    sigma = 8  # CHANGE ME

    # Smooth the image using a Gaussian kernel
    img_gaussian_filter = cv2.GaussianBlur(imggray, (0, 0), sigma)

    # Convolve the image with the Sobel operator (filter) to compute the numerical derivatives in the x and y directions
    sobelx = cv2.Sobel(img_gaussian_filter, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(img_gaussian_filter, cv2.CV_64F, 0, 1)

    # Compute the magnitude of the gradients
    Gmag = np.sqrt(sobelx * sobelx + sobely * sobely)

    threshold_left = 5  # CHANGE ME
    mask_mag_left = Gmag > threshold_left
    threshold_right = 25
    mask_mag_right = Gmag > threshold_right

    white_lower_hsv = np.array([15, 3, 166])  # CHANGE ME
    white_upper_hsv = np.array([179, 116, 255])  # CHANGE ME
    yellow_lower_hsv = np.array([10, 75, 50])        # CHANGE ME
    yellow_upper_hsv = np.array([35, 255, 255])  # CHANGE ME

    mask_white = cv2.inRange(imghsv, white_lower_hsv, white_upper_hsv)
    mask_yellow = cv2.inRange(imghsv, yellow_lower_hsv, yellow_upper_hsv)

    mask_left = np.ones(sobelx.shape)
    mask_left[:, int(np.floor(w / 2)) : w + 1] = 0
    mask_right = np.ones(sobelx.shape)
    mask_right[:, 0 : int(np.floor(w / 2))] = 0

    mask_sobelx_pos = sobelx > 0
    mask_sobelx_neg = sobelx < 0
    mask_sobely_neg = sobely < 0

    mask_left_edge = mask_ground * mask_left * mask_mag_left * mask_sobelx_neg * mask_sobely_neg * mask_yellow
    mask_right_edge = mask_ground * mask_right * mask_mag_right * mask_sobelx_pos * mask_sobely_neg * mask_white
    mask_lanes = cv2.bitwise_or(mask_left_edge, mask_right_edge)
    masked_image = cv2.bitwise_and(image, image, mask=mask_ground)
    display(Image(data=bgr8_to_jpeg(mask_lanes)))
    display(Image(data=bgr8_to_jpeg(image)))

    return mask_left_edge, mask_right_edge

In [ ]:
def get_steer_matrix_left_lane_markings(shape: Tuple[int, int]) -> np.ndarray:
    """
    Args:
        shape:              The shape of the steer matrix.

    Return:
        steer_matrix_left:  The steering (angular rate) matrix for Braitenberg-like control
                            using the masked left lane markings (numpy.ndarray)
    """

    width = int(shape[1] / 2)
    height = int(shape[0])

    steer_matrix_left_lane = np.zeros((height, shape[1]))

    iterable = (x for x in range(width))
    steer_matrix_left_lane_unit = np.fromiter(iterable, float)

    if steer_matrix_left_lane_unit.max() != 0:
        steer_matrix_left_lane_unit /= steer_matrix_left_lane_unit.max()

    steer_matrix_left_lane[:, :width] = np.tile(steer_matrix_left_lane_unit, (height, 1)) * -0.3
    steer_matrix_left_lane[int(height * 0.5) : -int(height * 0.2), int(width / 2) : width] = (
        steer_matrix_left_lane[int(height * 0.5) : -int(height * 0.2), int(width / 2) : width] * 2.0
    )

    return steer_matrix_left_lane

In [ ]:
while True:
    clear_output(wait=True)
    left, right = detect_lane_markings(camera.value)
    time.sleep(0.01)

# AELMIGER

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time

robot = Robot()
camera = Camera.instance()

In [ ]:
import numpy as np
import cv2
from scipy.optimize import lsq_linear
import CP_constants as const
from collections import deque

In [ ]:
"""

 Title: Lane Detection Algorithm
 Author: Anton Elmiger
 Created: 2020-05-26

 Information: Class that extracts a lane from an edge image
              and calculates the corresponding hyperbola-pair parameters

              Lane Model is described in this paper https://ieeexplore.ieee.org/abstract/document/1689679
              and in the wiki of github

"""



      
class Lane_Detection:
    def __init__(self):
        self.v = np.arange(0, const.IMAGE_HEIGHT, 1)  # vertical points
        self.u = np.arange(0, const.IMAGE_WIDTH, 1)  # horizontal points

        self.threshold = const.BOUNDARY_THRESH
        self.lane_width = const.LANE_WIDTH

        self.h = const.HORIZON # h-horizon height
        self.k = 0  # k-curvature of lane
        self.bl = 0 # b-skew of left lane
        self.br = 0 # b-skew of right lane
        self.bc = 0 # b-skew of lane center
        self.c = 0 # c-horizontal offset of lane

        self.bcdq = deque(maxlen=const.FILTER_STRENGTH)   
        self.bldq = deque(maxlen=const.FILTER_STRENGTH)   
        self.brdq = deque(maxlen=const.FILTER_STRENGTH)   

        self.left_lane_points = np.array([])
        self.right_lane_points = np.array([])
        
        self.lane = np.array([])

        # Bounds for the solving of hyperbola-pair parameters
        # [k,bl,br,c]
        # The constraint on c dramatically increases robustness
        self.low_b = np.array([-500000, -8, -8, const.IMAGE_WIDTH/2 -20])
        self.up_b = np.array([500000, 8, 8, const.IMAGE_WIDTH/2 +20])

    # Calculate lane hyperbola for given parameters
    def hyperbola_pair(self, b):
        return self.k/(self.v-self.h)+b*(self.v-self.h)+self.c

    # Function finds lane points in an edge image and classifies them to left and right lane
    # This function is used if no lane estimate exists, or the estimation is odd
    def get_initial_lane_points(self, edge_image):
        image_height = edge_image.shape[0]
        image_width = edge_image.shape[1]

        # initialize lane arrays
        left_lane_points = np.empty((image_height, 1))
        left_lane_points[:] = np.NAN
        right_lane_points = np.empty((image_height, 1))
        right_lane_points[:] = np.NAN

        lane_numbers = np.arange(image_width)
        edge_image = edge_image / 255

        for row in range(image_height-1, -1, -1):
            curr_row = np.multiply(
                (lane_numbers - image_height), edge_image[row, :])
            points_to_the_right = np.where(curr_row > 0)[0]
            points_to_the_left = np.where(curr_row < 0)[0]
            if points_to_the_right.size > 0:
                right_lane_points[row] = np.amin(points_to_the_right)
            if points_to_the_left.size > 0:
                left_lane_points[row] = np.amax(points_to_the_left)
            if row == 300:
                break
        self.left_lane_points = left_lane_points
        self.right_lane_points = right_lane_points

    # Function finds lane points in an edge image and classifies them to left and right lane
    def lane_points(self, edge_image):
        image_height = edge_image.shape[0]

        # initialize lane arrays
        left_lane_points = np.empty((image_height, 1))
        left_lane_points[:] = np.NAN
        right_lane_points = np.empty((image_height, 1))
        right_lane_points[:] = np.NAN

        # get the "bounding" lanes to filter outliers
        # only points between the bounds are considered inliers
        left_max_bound, left_min_bound, right_max_bound, right_min_bound = self.generate_bounding_lanes()

        # only considere points that are below the horizon (plus some extra space for robustness) if the horizon is in the image
        horizon_index = int(max(self.h+20,0))

        # get the 2D image position of edge pixels that are below the horizon index
        nonzero = cv2.findNonZero(edge_image[horizon_index:]).reshape((-1,2)).T
        # offset the Y-Coordinate by the horizon index
        nonzero[1] += horizon_index

        # classify all points in left bounding area as left lane points
        left_p = nonzero.T[(nonzero[0] < left_max_bound[nonzero[1]]) & (nonzero[0] > left_min_bound[nonzero[1]])]

        # classify all points in right bounding area as left right points
        # the flipping of the array is imortant for the next step
        right_p = np.flipud(nonzero.T[(nonzero[0] < right_max_bound[nonzero[1]]) & (nonzero[0] > right_min_bound[nonzero[1]])])

        # for each vertical row in the image that contains a left lane point ->
        # place the point that is closest the the centerline into the left lane points array
        np.put(left_lane_points,left_p[:,1],left_p[:,0])
        
        # for each vertical row in the image that contains a right lane point ->
        # place the point that is closest the the centerline into the right lane points array
        np.put(right_lane_points,right_p[:,1],right_p[:,0])

        self.left_lane_points = left_lane_points
        self.right_lane_points = right_lane_points


    # Function returns lane lines, that are left and right of the estimated lane lines
    # These bounding lines are then used to define an inlier area
    def generate_bounding_lanes(self):
        # horizontal points left lane
        left_max = self.hyperbola_pair(self.bl+(self.bc-self.bl)/self.threshold)
        # horizontal points left lane
        left_min = self.hyperbola_pair(self.bl-(self.bc-self.bl)/self.threshold)
        # horizontal points left lane
        right_max = self.hyperbola_pair(self.br+(self.bc-self.bl)/self.threshold)
        # horizontal points left lane
        right_min = self.hyperbola_pair(self.br-(self.bc-self.bl)/self.threshold)
        return left_max, left_min, right_max, right_min


    # Function solves for hyperbola-pair lane parameters
    # More info is in the paper listed at the top of this file
    def solve_lane(self):
        # generate matrices for lsq solver
        A, b = self.preprocess_for_solving()
        # returning the solved parameters (k,bl,br,c)
        self.solving_lane_params(A, b)

    def preprocess_for_solving(self):
        l = self.left_lane_points
        r = self.right_lane_points
        # following lines create A matrix  and b vector for least square porblem
        l_ind = ~np.isnan(l)
        r_ind = ~np.isnan(r)
        l_num = l[l_ind]
        r_num = r[r_ind]
        vl = self.v[l_ind.flatten()]
        vr = self.v[r_ind.flatten()]
        l_num = l_num.reshape((len(l_num), 1))
        r_num = r_num.reshape((len(r_num), 1))
        vl = vl.reshape(l_num.shape)
        vr = vr.reshape(r_num.shape)

        lh = (vl-self.h)
        lA = 1/lh
        rh = (vr-self.h)
        rA = 1/rh
        ones = np.ones(l_num.shape)
        zeros = np.zeros(l_num.shape)
        LA = np.hstack((np.hstack((lA, lh)), np.hstack((zeros, ones))))
        ones = np.ones(r_num.shape)
        zeros = np.zeros(r_num.shape)
        RA = np.hstack((np.hstack((rA, zeros)), np.hstack((rh, ones))))
        A = np.vstack((LA, RA))
        b = (np.concatenate((l_num, r_num))).flatten()
        return A, b

    def solving_lane_params(self, A, b):
        x = lsq_linear(A, b, bounds=(self.low_b, self.up_b), method='bvls', max_iter = 3).x
        # set new lane model param from least square solution
        self.k = x[0]
        self.bl=x[1]
        self.br=x[2]
        self.c = x[3]
        self.bc = (x[1]+x[2])/2
        # calc lane points
        self.lane = self.hyperbola_pair(self.bc)

    # function corrects false lane lines or missing lane lines
    def lane_sanity_checks(self,edge_image):
        #lane not found
        if self.k == 0:
            self.get_initial_lane_points(edge_image)
            self.solve_lane()

        #Only one lane found
        self.interpolate_missing_lane()            

        #Lane width not correct size
        self.adjust_lane_width()            

        #Vehicle not on lane -> recenter lane line
        self.recenter_lane()
    
        #smooth lane
        self.filter_lane()
         
        self.lane = self.hyperbola_pair(self.bc)

    def interpolate_missing_lane(self):
        #Only one lane found
        if ~np.isfinite(self.left_lane_points).any():
            self.bl = self.br-self.lane_width-0.3
            self.bc = (self.bl+self.br)/2            
        if ~np.isfinite(self.right_lane_points).any():
            self.br = self.bl+self.lane_width+0.3
            self.bc = (self.bl+self.br)/2            

    def adjust_lane_width(self):
        #Lane width not correct size
        if abs(self.bl-self.br)<(self.lane_width*0.8) or abs(self.bl-self.br)>(self.lane_width)*1.2:
            length_l = np.count_nonzero(~np.isnan(self.left_lane_points))
            length_r = np.count_nonzero(~np.isnan(self.right_lane_points))
            if length_l > length_r:
                self.br = self.bl+self.lane_width
            else:
                self.bl = self.br-self.lane_width
            self.bc = (self.bl+self.br)/2            

    def recenter_lane(self):
        #Vehicle not on lane -> recenter lane line
        if self.bc > (self.lane_width/1.1):
            self.bl=self.bl-self.lane_width
            self.br=self.br-self.lane_width

        if self.bc < (-self.lane_width/1.1):
            self.bl=self.bl+self.lane_width
            self.br=self.br+self.lane_width

    def filter_lane(self):
        self.bc = (self.bl+self.br)/2 
        self.bcdq.append(self.bc)
        self.bc = sum(bc for bc in self.bcdq)/len(self.bcdq)
        self.bldq.append(self.bl)
        self.bl = sum(bc for bc in self.bldq)/len(self.bldq)
        self.brdq.append(self.br)
        self.br = sum(bc for bc in self.brdq)/len(self.brdq)

In [ ]:
"""

 Title: Transform Points
 Author: Anton Elmiger
 Created: 2020-05-26

 Information: Class used to transform Points between camera space and world
              space
"""
import numpy as np
import CP_constants as const
from scipy.spatial.transform import Rotation as R


class Transform_Points:
    def __init__(self,_cam_angle=45):
        # cam angle from horizontal down in [degrees]]
        cam_angle = _cam_angle

        self.rot_mat_inv = R.from_euler(
            'xyz', [-90-cam_angle, 0, -90], degrees=True).as_matrix()
        self.rot_mat = np.linalg.inv(self.rot_mat_inv)

        # fov = 160 #fov in degrees/2
        width = const.IMAGE_WIDTH
        height = const.IMAGE_HEIGHT
        aspect_ratio = width / height
        fx = 417/2  # width / (np.tan(np.radians(fov) / 2) * 2)
        fy = fx
        self.cameraMatrix = np.array(
            [[fx, 0, width / 2], [0, fy, height / 2], [0, 0, 1]])
        self.cameraMatrixInv = np.linalg.inv(self.cameraMatrix)
        self.tt = -np.array([[0.182], [0.], [0.195]])
        self.rotationMatrix = self.rot_mat  # np.empty([3, 3])
        self.tvec = self.rotationMatrix @ self.tt
        self.rotationMatrixInv = np.linalg.inv(self.rotationMatrix)

        self.poly_koeff = np.array([0.0, 0.0, 0.0])

    def imagePoint_to_worldPoint(self, imgPoints):
        imgPoints = imgPoints.T
        n, m = imgPoints.shape

        imgPoints = np.vstack([imgPoints, np.ones((1, m))])
        leftSideMat = self.rotationMatrixInv.dot(
            self.cameraMatrixInv).dot(imgPoints)
        rightSideMat = self.rotationMatrixInv.dot(self.tvec)
        s = (0 + rightSideMat[2, 0])/leftSideMat[2, :]
        return self.rotationMatrixInv.dot(s*self.cameraMatrixInv.dot(imgPoints)-self.tvec)

    def worldPoint_to_imagePoint(self, worldPoint):
        worldPoint = worldPoint.reshape(-1, 1)
        rightSideMat = self.cameraMatrix.dot(
            self.rotationMatrix.dot(worldPoint)+self.tvec)
        return np.round((rightSideMat/rightSideMat[2, 0])[0:2])

    def transform_lane_to_poly(self, lane_class):
        lane_points = np.hstack((lane_class.lane.reshape(
            (-1, 1)), lane_class.v.reshape((-1, 1))))
        lane_points = lane_points[int(max(const.HORIZON+20, 20)):, :]
        worldCoord = self.imagePoint_to_worldPoint(lane_points).T
        self.poly_koeff = np.polyfit(worldCoord[:, 0], worldCoord[:, 1], 2)

    def send_lane_tcp(self, conn):
        conn.sendall(self.poly_koeff.astype(np.float64).tobytes())

In [ ]:

"""

 Title: Visualization Class
 Author: Anton Elmiger
 Created: 2020-05-21

 Information: Class to visualize the opencv images

"""
import numpy as np
import cv2
import CP_constants as const
import math


class Visualization:
        def __init__(self):
            self.array_of_imgs = []
            self.vidArray = []
        
        def clear_imgs(self):
            self.array_of_imgs = []

        def append_img(self,img):
            res_img = cv2.resize(img, (640, 480))
            if len(img.shape) == 2:
                res_img = cv2.cvtColor(res_img, cv2.COLOR_GRAY2RGB)
            self.array_of_imgs.append(res_img)

        def draw_lane_lines(self,img,lane_class):
            lane_c = lane_class.hyperbola_pair(lane_class.bc)
            lane_r = lane_class.hyperbola_pair(lane_class.br)
            lane_l = lane_class.hyperbola_pair(lane_class.bl)

            draw_points_c = (np.asarray([lane_c, lane_class.v]).T).astype(np.int32)
            draw_points_l = (np.asarray([lane_l, lane_class.v]).T).astype(np.int32)
            draw_points_r = (np.asarray([lane_r, lane_class.v]).T).astype(np.int32)

            cv2.polylines(img, [draw_points_c[int(max(const.HORIZON+20,30)):]], False, (0,255,0),10)
            cv2.polylines(img, [draw_points_l[int(max(const.HORIZON+20,30)):]], False, (255,0,0),4)
            cv2.polylines(img, [draw_points_r[int(max(const.HORIZON+20,30)):]], False, (0,0,255),4) 
            return img
        
        def draw_lane_points(self,img,lane_class):
            draw_points_l = (np.asarray([lane_class.left_lane_points.reshape(-1,), lane_class.v]).T).astype(np.int32)
            for point in draw_points_l:
                img = cv2.circle(img,tuple(point),8,(255,0,0),-1)

            draw_points_r = (np.asarray([lane_class.right_lane_points.reshape(-1,), lane_class.v]).T).astype(np.int32)
            for point in draw_points_r:
                img = cv2.circle(img,tuple(point),8,(0,0,255),-1)
                            

        def show_imgs(self):
            n_images = len(self.array_of_imgs)
            rows = math.floor(n_images/3)+1
            row_imgs = self.array_of_imgs[0]
            for i in range(n_images-1):
                row_imgs = np.hstack((row_imgs, self.array_of_imgs[i+1]))
            cv2.imshow("Imgs", row_imgs)
            cv2.waitKey(1)
            if const.WRITE_VIDEO:
                self.vidArray.append(row_imgs)

        def write_video(self):
            height, width, layers = self.vidArray[0].shape
            size = (width,height)

            out = cv2.VideoWriter('vidOut.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
            for i in range(len(self.vidArray)):
                out.write(self.vidArray[i])
            out.release()


In [ ]:
def edge_detection(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float)       # convert to gray
    mask_ground = np.ones(img.shape, dtype=np.uint8)
    mask_ground = np.zeros(img.shape, dtype=np.uint8)
    mask_ground[int(img.shape[0] - const.HORIZON) :, :] = 1
    blurredGray = cv2.blur(gray*mask_ground, (3, 3))                                # blur img for robustness
    blurredSobelImg = cv2.Sobel(blurredGray, cv2.CV_8U, 1, 0, ksize=1)  # calculate sobel gradient
    ret, threshSobel = cv2.threshold(                                   # soebel img to binary by threshold
        blurredSobelImg, 7, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)                                  # erode image to remove noise
    erodedSobel = cv2.erode(threshSobel, kernel, iterations=1)          # erode image to remove noise
    return erodedSobel

In [ ]:
visual = Visualization()
ld = Lane_Detection()
tp = Transform_Points()

img = np.array(camera.value)

edge_image = edge_detection(img)            # Detect Edges in image

ld.get_initial_lane_points(edge_image)      # Apply Lane initialization on first image
ld.solve_lane()                             # Solve lane model parameters from lane points
ld.lane_sanity_checks(edge_image)           # Apply corrections to lane model
tp.transform_lane_to_poly(ld)               # Calculate polynomial coeff from lane model

visual.draw_lane_lines(img, ld)                             # Draw lane model lines on image
edge_image = cv2.cvtColor(edge_image, cv2.COLOR_GRAY2RGB)   # Generate Edge Image
visual.draw_lane_points(edge_image, ld)                     # Draw lane edge points on image
display(Image(data=bgr8_to_jpeg(img)))                      # Display img with drawing
display(Image(data=bgr8_to_jpeg(edge_image)))                      # Display img with drawing


In [ ]:
while const.VISUALIZE:
       clear_output(wait = True)
       img = np.array(camera.value)

       for i in range(2):                      # Solve for lane model multiple times for convergence
              ld.lane_points(edge_image)      
              ld.solve_lane()                     # Solve lane model parameters from lane points
              ld.lane_sanity_checks(edge_image)   # Apply corrections to lane model
       tp.transform_lane_to_poly(ld)           # Calculate polynomial coeff from lane model

       # visual.clear_imgs()                                         # Clear Images to show every iteration
       visual.draw_lane_lines(img, ld)                             # Draw lane model lines on image
       edge_image = cv2.cvtColor(edge_image, cv2.COLOR_GRAY2RGB)   # Generate Edge Image
       visual.draw_lane_points(edge_image, ld)                     # Draw lane edge points on image
       display(Image(data=bgr8_to_jpeg(img)))                      # Append img for drawing
       # visual.append_img(edge_image)                               # Append img for drawing
       # img = vid.bev(img)                                          # Birds Eye View transformation
       # visual.append_img(img)                                      # Append img for drawing
       # visual.show_imgs()                                          # Show images that were appended for drawing